<a href="https://colab.research.google.com/github/lmgrj/Deep-Learning-avec-PyTorch/blob/main/TP_N%C2%B0_3_DEEP_LEARNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1>WISD&EID2/Deep Learning</center></h1> 

<center><h1>TP 3 </center><h1>

<center><h1>Adaline , Perceptron and Multi layer Perceptron</center><h1>


---
By: **Lamgarraj Mohamed**   &   **Yazid Tagnaouti Moumnani**


---

# Deep Learning
## Practical Deep Learning Tutorial with PyTorch - Tutorial N° 3

### 2020-2021

# Importing necessary libraries


In [ ]:
import torch
from torch.autograd import grad
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_blobs
from torch import nn

# Adaline

1. Built ADALINE model using the nn.Module class 

In [ ]:
class Adaline(torch.nn.Module):
    def __init__(self, num_features):
        super(Adaline, self).__init__()
        self.linear = nn.Linear(num_features,1)                                                                                                       
        self.linear.weight.detach().zero_()            
        self.linear.bias.detach().zero_()              
    
    def forward(self, x):
        activations = self.linear(x)
        return activations.view(-1)
    

2. Using 'iris.txt', create a binary datasets in 2-D : The last 100 instances of iris described only by the 2nd and 3rd features
    
    Split the dataset into traing and test sets (70%,30%) 

    Normalize the dataset

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
path='/content/drive/My Drive/deeplearning/iris.txt'

df = pd.read_csv(path, index_col=None, header=None)    #reding iris
df.columns = ['x1', 'x2', 'x3', 'x4', 'y'] #define features names
df = df.iloc[50:150]   #taking last 100 elements that belong to 2 classes (100*2)
df['y'] = df['y'].apply(lambda x: 0 if x == 'Iris-versicolor' else 1) #coder la 2e classe par 0 la 3e par 1


# Assign features and target

X = torch.tensor(df[['x2', 'x3']].values, dtype=torch.float) #taking just last 2 features
y = torch.tensor(df['y'].values, dtype=torch.int) # y is a tensor of features

# Shuffling & train/test split

torch.manual_seed(123)
shuffle_idx = torch.randperm(y.size(0), dtype=torch.long)  #shuffle indexes
X, y = X[shuffle_idx], y[shuffle_idx]  #shuffle data (we shuffle also labels)
percent70 = int(shuffle_idx.size(0)*0.7)
X_train, X_test = X[shuffle_idx[:percent70]], X[shuffle_idx[percent70:]]  # 70% for training training 
y_train, y_test = y[shuffle_idx[:percent70]], y[shuffle_idx[percent70:]]  # 30% for  testing

# Normalize (mean zero, unit variance)

mu, sigma = X_train.mean(dim=0), X_train.std(dim=0)
X_train = (X_train - mu )/sigma
X_test = (X_test - mu )/sigma

Mounted at /content/drive


3. Train the model : we will use MSELoss (mean squared error (squared L2 norm)) as loss function. The optimizer is SGD (Stochastic Gradient Descent) with learning rate 0.01.

In [ ]:
def train(model, x, y, epochs, learning_rate, seed):
    cost = []
    torch.manual_seed(seed)
    optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)   #use a SGD optimizer
    c = nn.MSELoss()
    for e in range(epochs):
        yhat = model.forward(x) # calcul yhat
        loss = c(yhat,y) # calcul the loss function using MSE
        optimizer.zero_grad()    # set the gradients to zero
        loss.backward()  # calculer le gradients
        optimizer.step()  # mise a jour des poids ####                        
        print('epoch {} , loss {}'.format(e,loss.item()))  

In [ ]:
model = Adaline(num_features=X_train.size(1))
train(model, X_train, y_train.float(),num_epochs=100,learning_rate=0.01,seed=100)

epoch 0 , loss 0.48571428656578064
epoch 1 , loss 0.4699629247188568
epoch 2 , loss 0.45489606261253357
epoch 3 , loss 0.4404822289943695
epoch 4 , loss 0.42669153213500977
epoch 5 , loss 0.4134954810142517
epoch 6 , loss 0.40086689591407776
epoch 7 , loss 0.3887799382209778
epoch 8 , loss 0.37721002101898193
epoch 9 , loss 0.3661336600780487
epoch 10 , loss 0.35552868247032166
epoch 11 , loss 0.3453736901283264
epoch 12 , loss 0.33564847707748413
epoch 13 , loss 0.3263336420059204
epoch 14 , loss 0.317410945892334
epoch 15 , loss 0.30886268615722656
epoch 16 , loss 0.30067235231399536
epoch 17 , loss 0.29282382130622864
epoch 18 , loss 0.2853020429611206
epoch 19 , loss 0.27809247374534607
epoch 20 , loss 0.2711813747882843
epoch 21 , loss 0.26455551385879517
epoch 22 , loss 0.2582024037837982
epoch 23 , loss 0.2521100640296936
epoch 24 , loss 0.24626705050468445
epoch 25 , loss 0.24066250026226044
epoch 26 , loss 0.23528605699539185
epoch 27 , loss 0.2301277220249176
epoch 28 , loss 

4. Compute the model accuracy 

In [ ]:
def custom_where(cond, x_1, x_2):
    return (cond * x_1) + (torch.logical_not(cond) * x_2)
train_pred = model.forward(X_train)
train_acc = torch.mean(
    (custom_where(train_pred > 0.5, 1, 0).int() == y_train).float())
test_pred = model.forward(X_test)
test_acc = torch.mean((custom_where(test_pred > 0.5, 1, 0).int() == y_test).float())
print('Training Accuracy:    %.2f' % (train_acc*100))
print('Test Accuracy:    %.2f' % (test_acc*100))
print('Weights:   ', model.linear.weight)
print('Bias:   ', model.linear.bias)

Training Accuracy: 87.14
Test Accuracy: 76.67
Weights Parameter containing:
tensor([[0.0043, 0.3283]], requires_grad=True)
Bias Parameter containing:
tensor([0.4213], requires_grad=True)


# Perceptron

5. Built a Perceptron model using nn.Module class

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Perceptron():
    def __init__(self, num_features):
        self.num_features = num_features
        self.weights = torch.zeros(num_features, 1, 
                                   dtype=torch.float32, device=device)
        self.bias = torch.zeros(1, dtype=torch.float32, device=device)
        self.ones = torch.ones(1)
        self.zeros = torch.zeros(1)

    def forward(self, x):
        linear = torch.add(torch.mm(x, self.weights), self.bias)
        predictions = custom_where(linear > 0., 1, 0).float()
        return predictions
        
    def backward(self, x, y):  
        predictions = self.forward(x)
        errors = y - predictions
        return errors
        
        
    def train(self, x, y, epochs):
        for e in range(epochs):
            for i in range(y.shape[0]):
                errors = self.backward(x[i].view(1, self.num_features), y[i]).view(-1)
                self.weights += (errors * x[i]).view(self.num_features, 1)
                self.bias += errors
                
    def evaluate(self, x, y):
        predictions = self.forward(x).reshape(-1)
        accuracy = torch.sum(predictions == y).float() / y.shape[0]
        return accuracy

6. Load the 'perceptron_toydata' dataset

    Split the dataset into train and test sets
    
    Normalize the data

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
path='/content/drive/My Drive/deeplearning/perceptron_toydata.txt'

df = pd.read_csv(path, index_col=None, header=None , delimiter='\t')
df.columns = ['x1', 'x2', 'y']
X = torch.tensor(df[['x1', 'x2']].values, dtype=torch.float) 
y = torch.tensor(df['y'].values, dtype=torch.int) 
print('Class label counts:', torch.bincount(y))
print('X.shape:', X.shape)
print('y.shape:', y.shape)

# Shuffling & train/test split
shuffle_idx = torch.randperm(y.size(0), dtype=torch.long)
X, y = X[shuffle_idx], y[shuffle_idx]
percent70 = int(shuffle_idx.size(0)*0.7)
X_train, X_test = X[shuffle_idx[:percent70]], X[shuffle_idx[percent70:]]
y_train, y_test = y[shuffle_idx[:percent70]], y[shuffle_idx[percent70:]]
# Normalize (mean zero, unit variance)
mu, sigma = X_train.mean(axis=0), X_train.std(axis=0)
X_train = (X_train - mu) / sigma
X_test = (X_test - mu) / sigma
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Class label counts: tensor([50, 50])
X.shape: torch.Size([100, 2])
y.shape: torch.Size([100])
torch.Size([70, 2])
torch.Size([70])
torch.Size([30, 2])
torch.Size([30])


7. Train the perceptron

In [ ]:
model = Perceptron(num_features=2)

Xtrain = torch.tensor(X_train, dtype=torch.float32, device=device)
ytrain = torch.tensor(y_train, dtype=torch.float32, device=device)

model.train(Xtrain, ytrain, epochs=5)

print('Model parameters:')
print('  Weights: %s' % model.weights)
print('  Bias: %s' % model.bias)

Model parameters:
  Weights: tensor([[2.4768],
        [0.8986]])
  Bias: tensor([-1.])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


8. evaluate the model (accuracy)

In [ ]:
Xtest = torch.tensor(X_test, dtype=torch.float32, device=device)
ytest = torch.tensor(y_test, dtype=torch.float32, device=device)

test_acc = model.evaluate(Xtest, ytest)
print('Test set accuracy: %.2f%%' % (test_acc*100))

Test set accuracy: 100.00%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


# Multi Layer Perceptron

Unlike the single-layer perceptron, the Multi Layer Perceptron models have hidden layers
between the input and the output layers. After every hidden layer, an activation function 
is applied to introduce non-linearity. 

9. Built a simple Multi Layer Perceptron model withe one hidden layer. 
After the hidden layer, we will use ReLU as activation before the information is sent to the output layer.
As an output activation function, we will use Sigmoid. 

In [ ]:
class MultilayerPerceptron(torch.nn.Module):
    def __init__(self, num_features,num_hidden_1):
        super(MultilayerPerceptron, self).__init__()
        self.linear_1 = torch.nn.Linear(num_features, num_hidden_1)
        self.linear_out = torch.nn.Linear(num_hidden_1, 1)

        

    def forward(self, x):
        out = self.linear_1(x)
        out = F.relu(out)
        out = self.linear_out(out)
        out = F.sigmoid(out)
        
        return out

10. Create a random datasets and assign binary labels {0,1}

In [ ]:
def blob_label(y, label, loc): # assign labels
    target = np.copy(y)
    for l in loc:
        target[y == l] = label
    return target
x_train, y_train = make_blobs(n_samples=40, n_features=2, cluster_std=1.5, shuffle=True)
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(blob_label(y_train, 0, [0]))
y_train = torch.FloatTensor(blob_label(y_train, 1, [1,2,3]))
x_test, y_test = make_blobs(n_samples=10, n_features=2, cluster_std=1.5, shuffle=True)
x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(blob_label(y_test, 0, [0]))
y_test = torch.FloatTensor(blob_label(y_test, 1, [1,2,3]))
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

torch.Size([40, 2])
torch.Size([40])
torch.Size([10, 2])
torch.Size([10])


11. Define the model with input dimension 2 and hidden dimension 10. 
Since the task is to classify binary labels, we can use BCELoss (Binary Cross Entropy Loss) as loss function.
The optimizer is SGD (Stochastic Gradient Descent) with learning rate 0.01.

In [ ]:
model = MultilayerPerceptron(2, 10)
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)  #use a SGD optimizer
criterion = torch.nn.MSELoss() 

12. Check the test loss before the model training and compare it with the test loss after the training.

In [ ]:
model.eval()
y_pred = model(x_test)
before_train = criterion(y_pred.squeeze(), y_test)
print('Test loss before training' , before_train.item())

Test loss before training 0.23654893040657043


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
model.train()
epoch = 20
for epoch in range(epoch):
    optimizer.zero_grad()
    y_pred = model.forward(x_train) 
    loss = criterion(y_pred,y_train)
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Backward pass
    loss.backward()
    optimizer.step()

Epoch 0: train loss: 0.2857312858104706
Epoch 1: train loss: 0.28241994976997375
Epoch 2: train loss: 0.27909523248672485
Epoch 3: train loss: 0.2757806181907654
Epoch 4: train loss: 0.27250081300735474
Epoch 5: train loss: 0.26928097009658813
Epoch 6: train loss: 0.26614561676979065
Epoch 7: train loss: 0.2631177306175232
Epoch 8: train loss: 0.26021769642829895
Epoch 9: train loss: 0.2574624717235565
Epoch 10: train loss: 0.25486528873443604
Epoch 11: train loss: 0.25243523716926575
Epoch 12: train loss: 0.2501772344112396
Epoch 13: train loss: 0.24809250235557556
Epoch 14: train loss: 0.2461787760257721
Epoch 15: train loss: 0.2444310337305069
Epoch 16: train loss: 0.24284198880195618
Epoch 17: train loss: 0.24140284955501556
Epoch 18: train loss: 0.24010413885116577
Epoch 19: train loss: 0.23894067108631134


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([40])) that is different to the input size (torch.Size([40, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
y_pred = model(x_test)
before_train = criterion(y_pred.squeeze(), y_test)
print('Test loss after training' , before_train.item())

13. In order to improve the model, you can try out different parameter values for your
hyperparameters(ie. hidden dimension size, epoch size, learning rates). You can also 
try changing the structure of your model (ie. adding more hidden layers) to see if your
mode improves. 

In [ ]:
class MultilayerPerceptronDifferent(torch.nn.Module):

    def __init__(self, num_features,num_hidden_1,num_hidden_2):
        super(MultilayerPerceptronDifferent, self).__init__()
        self.linear_1 = torch.nn.Linear(num_features, num_hidden_1)
        self.linear_2 = torch.nn.Linear(num_hidden_1, num_hidden_2)
        self.linear_out = torch.nn.Linear(num_hidden_2, 1)

    def forward(self, x):
        out = self.linear_1(x)
        out = F.relu(out)
        out = self.linear_2(out)
        out = F.relu(out)
        out = self.linear_out(out)
        out = F.sigmoid(out)
        return out


model1 = MultilayerPerceptronDifferent(2,512,256)
optimizer = torch.optim.SGD(model.parameters(),lr=0.01) 
criterion = torch.nn.MSELoss()

In [ ]:
model1.train()
epoch = 30
for epoch in range(epoch):
    optimizer.zero_grad()
    y_pred = model.forward(x_train) 
    loss = criterion(y_pred,y_train)
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Backward pass
    loss.backward()
    optimizer.step()

model1.eval()
y_pred = model1(x_test)
after_train = criterion(y_pred.squeeze(), y_test) 
print('Test loss after Training' , after_train.item())

Epoch 0: train loss: 0.23789472877979279
Epoch 1: train loss: 0.23695605993270874
Epoch 2: train loss: 0.2361147552728653
Epoch 3: train loss: 0.23536136746406555
Epoch 4: train loss: 0.2346871942281723
Epoch 5: train loss: 0.2340841293334961
Epoch 6: train loss: 0.23354469239711761
Epoch 7: train loss: 0.23306213319301605
Epoch 8: train loss: 0.2326304018497467
Epoch 9: train loss: 0.23224399983882904
Epoch 10: train loss: 0.2318979650735855
Epoch 11: train loss: 0.23158787190914154
Epoch 12: train loss: 0.2313098907470703
Epoch 13: train loss: 0.23106049001216888
Epoch 14: train loss: 0.23083651065826416
Epoch 15: train loss: 0.23063524067401886
Epoch 16: train loss: 0.2304542511701584
Epoch 17: train loss: 0.23029132187366486
Epoch 18: train loss: 0.23014454543590546
Epoch 19: train loss: 0.2300122082233429
Epoch 20: train loss: 0.22989274561405182
Epoch 21: train loss: 0.22978487610816956
Epoch 22: train loss: 0.22968733310699463
Epoch 23: train loss: 0.22959907352924347
Epoch 24: 

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([40])) that is different to the input size (torch.Size([40, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
